# P2 MAAAI 

PREPROCESADO DE DATOS:

In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import numpy as np

In [7]:

# Cargar el conjunto de datos CIFAR-100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

print(f"Entrenamiento: {x_train.shape}, Prueba: {x_test.shape}")

# Eliminamos el 80% de las etiquetas en el conjunto de entrenamiento
np.random.seed(42)
unlabeled_indices = np.random.choice(len(x_train), size=int(0.8 * len(x_train)), replace=False)
labeled_indices = np.setdiff1d(np.arange(len(x_train)), unlabeled_indices)

# Creamos las particiones de datos
x_train_labeled = x_train[labeled_indices]
y_train_labeled = y_train[labeled_indices]
x_train_unlabeled = x_train[unlabeled_indices]
y_train_unlabeled = y_train[unlabeled_indices]  

print(f"Instancias de entrenamiento etiquetadas: {x_train_labeled.shape}, No etiquetadas: {x_train_unlabeled.shape}")


Entrenamiento: (50000, 32, 32, 3), Prueba: (10000, 32, 32, 3)
Instancias de entrenamiento etiquetadas: (10000, 32, 32, 3), No etiquetadas: (40000, 32, 32, 3)


CREACIÓN DE LA RED NEURONAL CONVOLUCIONAL

In [ ]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [52]:

def crear_modelo():
    model = models.Sequential()

    # Bloque convolucional 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005), input_shape=(32, 32, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Bloque convolucional 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                             kernel_regularizer=regularizers.l2(0.00005)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    # Aplanar
    model.add(layers.Flatten())

    # Bloque denso 1
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.6))

    # Capa de salida
    model.add(layers.Dense(100, activation='softmax'))

    # Compilación del modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model



In [53]:

# Crear modelo
model = crear_modelo()

# Callbacks para mejorar el entrenamiento
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
]

# Entrenamiento del modelo
history = model.fit(
    x_train_labeled, y_train_labeled,
    epochs=20,
    batch_size=256,
    validation_data=(x_test, y_test),
    callbacks=callbacks
)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 31s 635ms/step - accuracy: 0.0273 - loss: 5.4931 - val_accuracy: 0.0285 - val_loss: 6.4452
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 457ms/step - accuracy: 0.0803 - loss: 4.5391 - val_accuracy: 0.0618 - val_loss: 4.3131
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 387ms/step - accuracy: 0.1172 - loss: 4.1092 - val_accuracy: 0.1383 - val_loss: 3.8971
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 16s 410ms/step - accuracy: 0.1580 - loss: 3.7926 - val_accuracy: 0.1773 - val_loss: 3.6741
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 475ms/step - accuracy: 0.1875 - loss: 3.5513 - val_accuracy: 0.1803 - val_loss: 3.6116
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 477ms/step - accuracy: 0.2326 - loss: 3.2675 - val_accuracy: 0.2147 - val_loss: 3.4498
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 457ms/step - accuracy: 0.2708 - loss: 3.0363 - val_accuracy: 0.2405 - val_loss: 3.2954
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 17s 418ms/step - accuracy: 0.3183 - loss: 2.7917 - val_accu

In [54]:
# Evaluamos el rendimiento en el conjunto de prueba
train_loss, train_acc = model.evaluate(x_train_labeled, y_train_labeled, verbose=0)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Self-training: Train Accuracy: {:.4f} - Test Accuracy: {:.4f}".format(train_acc, test_acc))


313/313 - 6s - 19ms/step - accuracy: 0.2961 - loss: 2.9722
Self-training: Train Accuracy: 0.7229 - Test Accuracy: 0.2961
